In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

In [7]:
prices = pd.read_excel(r"C:\Users\benji\Downloads\Pricing_Oil_Gas_NGL.xlsx")
production = pd.read_excel(r"C:\Users\benji\Downloads\Production_Oil_Gas_NGL.xlsx")
production

,Date,Oil_Prod,NGL_Prod,Gas_Prod
0,2014-08-01,16778.787645,1095.658543,5681.192447
1,2014-09-01,12654.595199,840.427184,4357.770582
2,2014-10-01,10261.173695,693.084143,3593.769631
3,2014-11-01,8682.106765,596.418602,3092.540899
4,2014-12-01,7555.366259,527.860023,2737.051973
...,...,...,...,...
288,2038-08-01,327.092701,31.769343,164.729925
289,2038-09-01,324.827784,31.549359,163.589271
290,2038-10-01,322.578545,31.330898,162.456508
291,2038-11-01,320.344894,31.113952,161.331605


In [8]:
merged_data = pd.merge(prices, production, how='outer', on='Date')
merged_data['Oil_Revenue'] = merged_data['Oil_Price'] * merged_data['Oil_Prod']
merged_data['NGL_Revenue'] = merged_data['NGL_Price'] * merged_data['NGL_Prod']
merged_data['Gas_Revenue'] = merged_data['Gas_Price'] * merged_data['Gas_Prod']
merged_data

,Date,Oil_Price,NGL_Price,Gas_Price,Oil_Prod,NGL_Prod,Gas_Prod,Oil_Revenue,NGL_Revenue,Gas_Revenue
0,2014-08-01,100.0776,37.0720,4.446,16778.787645,1095.658543,5681.192447,1.679181e+06,40618.253520,25258.581620
1,2014-09-01,100.1052,37.1595,4.468,12654.595199,840.427184,4357.770582,1.266791e+06,31229.853933,19470.518961
2,2014-10-01,99.2128,36.8970,4.457,10261.173695,693.084143,3593.769631,1.018040e+06,25572.725629,16017.431246
3,2014-11-01,98.2284,36.5330,4.459,8682.106765,596.418602,3092.540899,8.528295e+05,21788.960787,13789.639870
4,2014-12-01,97.2348,36.1795,4.496,7555.366259,527.860023,2737.051973,7.346445e+05,19097.711716,12305.785671
...,...,...,...,...,...,...,...,...,...,...
288,2038-08-01,82.4412,30.3765,5.067,327.092701,31.769343,164.729925,2.696591e+04,965.041439,834.686531
289,2038-09-01,82.4412,30.3765,5.067,324.827784,31.549359,163.589271,2.677919e+04,958.359117,828.906837
290,2038-10-01,82.4412,30.3765,5.067,322.578545,31.330898,162.456508,2.659376e+04,951.723022,823.167126
291,2038-11-01,82.4412,30.3765,5.067,320.344894,31.113952,161.331605,2.640962e+04,945.132976,817.467243


In [9]:
nri = 0.75 #Net Revenue Interest 
merged_data['Oil_Rev_NRI'] = merged_data['Oil_Price'] * merged_data['Oil_Prod']
merged_data['NGL_Rev_NRI'] = merged_data['NGL_Price'] * merged_data['NGL_Prod']
merged_data['Gas_Rev_NRI'] = merged_data['Gas_Price'] * merged_data['Gas_Prod']
merged_data['Total_Revenue'] = merged_data['Oil_Rev_NRI'] + merged_data['NGL_Rev_NRI'] + merged_data['Gas_Rev_NRI']
merged_data[['Date','Oil_Rev_NRI','NGL_Rev_NRI','Gas_Rev_NRI','Total_Revenue']]

,Date,Oil_Rev_NRI,NGL_Rev_NRI,Gas_Rev_NRI,Total_Revenue
0,2014-08-01,1.679181e+06,40618.253520,25258.581620,1.745058e+06
1,2014-09-01,1.266791e+06,31229.853933,19470.518961,1.317491e+06
2,2014-10-01,1.018040e+06,25572.725629,16017.431246,1.059630e+06
3,2014-11-01,8.528295e+05,21788.960787,13789.639870,8.884081e+05
4,2014-12-01,7.346445e+05,19097.711716,12305.785671,7.660480e+05
...,...,...,...,...,...
288,2038-08-01,2.696591e+04,965.041439,834.686531,2.876564e+04
289,2038-09-01,2.677919e+04,958.359117,828.906837,2.856646e+04
290,2038-10-01,2.659376e+04,951.723022,823.167126,2.836865e+04
291,2038-11-01,2.640962e+04,945.132976,817.467243,2.817222e+04


In [10]:
LOE = 9500
Ad_Valorem_Tax = 0.025
Severance_Tax_Oil = 0.046
Severance_Tax_NGL = 0.075
Severance_Tax_Gas = 0.075

merged_data['LOE'] = [LOE for i in range(293)]
merged_data['Ad_Valorem_Tax'] = [Ad_Valorem_Tax * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data['Sev_Tax_Oil'] = [Severance_Tax_Oil * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data['Sev_Tax_NGL'] = [Severance_Tax_NGL * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data['Sev_Tax_Gas'] = [Severance_Tax_Gas * merged_data['Total_Revenue'][i] for i in range(293)]
merged_data[['Date','LOE','Ad_Valorem_Tax','Sev_Tax_Oil','Sev_Tax_NGL','Sev_Tax_Gas']]

,Date,LOE,Ad_Valorem_Tax,Sev_Tax_Oil,Sev_Tax_NGL,Sev_Tax_Gas
0,2014-08-01,9500,43626.440838,80272.651142,130879.322515,130879.322515
1,2014-09-01,9500,32937.278905,60604.593186,98811.836716,98811.836716
2,2014-10-01,9500,26490.748262,48742.976803,79472.244787,79472.244787
3,2014-11-01,9500,22210.201421,40866.770614,66630.604263,66630.604263
4,2014-12-01,9500,19151.200612,35238.209126,57453.601835,57453.601835
...,...,...,...,...,...,...
288,2038-08-01,9500,719.141068,1323.219565,2157.423204,2157.423204
289,2038-09-01,9500,714.161456,1314.057079,2142.484368,2142.484368
290,2038-10-01,9500,709.216313,1304.958015,2127.648938,2127.648938
291,2038-11-01,9500,704.305443,1295.922014,2112.916328,2112.916328


In [11]:
merged_data['D&CCAPEX'] = [0 for i in range(293)]
merged_data['D&CCAPEX'].iloc[0] = 6159101
merged_data['Cash Flow'] = merged_data['Total_Revenue'] - merged_data['D&CCAPEX'] - merged_data['Ad_Valorem_Tax'] - merged_data['Sev_Tax_Oil'] - merged_data['Sev_Tax_NGL'] - merged_data['Sev_Tax_Gas']
merged_data['Cumulative Cash Flows'] = np.cumsum(merged_data['Cash Flow'])
merged_data[['Date','D&CCAPEX','Cash Flow','Cumulative Cash Flows']]

C:\Users\benji\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Date,D&CCAPEX,Cash Flow,Cumulative Cash Flows
0,2014-08-01,6159101,-4.799701e+06,-4.799701e+06
1,2014-09-01,0,1.026326e+06,-3.773375e+06
2,2014-10-01,0,8.254517e+05,-2.947924e+06
3,2014-11-01,0,6.920699e+05,-2.255854e+06
4,2014-12-01,0,5.967514e+05,-1.659102e+06
...,...,...,...,...
288,2038-08-01,0,2.240844e+04,2.589366e+07
289,2038-09-01,0,2.225327e+04,2.591591e+07
290,2038-10-01,0,2.209918e+04,2.593801e+07
291,2038-11-01,0,2.194616e+04,2.595996e+07


In [12]:
merged_data['Cash Flow'].sum()

25981749.764408834

In [13]:
cash_flow = merged_data['Cash Flow'].values
merged_data['Time'] = [i for i in range(293)]
t = merged_data['Time'].values

def npv(irr, cashfs, t):  
    x =  np.sum(cashfs / (1 + irr) ** t)
    return round(x, 3)

def irr(cashfs, t, x0, **kwargs):
    y = (fsolve(npv, x0=x0, args=(cashfs, t), **kwargs)).item()
    return round(y*100, 4)
    
def payback():
    final_full_year = merged_data[merged_data['Cumulative Cash Flows'] < 0].index.values.max()
    fractional_yr = -merged_data['Cumulative Cash Flows'][final_full_year]/merged_data['Cash Flow'][final_full_year + 1]
    period = final_full_year + fractional_yr
    return round(period, 1)

def moic():
    z = merged_data['Cash Flow'].sum()/merged_data['D&CCAPEX'].sum()
    return round(z,1)

print("NPV : $", npv(irr=0.10, cashfs=cash_flow, t=t))
print("IRR : ", irr(cashfs=cash_flow, t=t, x0=0.10), "%")
print("Payback Period : ", payback(), "Months")
print("MOIC : ", moic(), "x")

NPV : $ 710608.99
IRR :  12.1944 %
Payback Period :  7.0 Months
MOIC :  4.2 x
